## chapter_06_SQL高级处理

### 1. 窗口函数

窗口函数也称为**OLAP函数**。OLAP 是 **OnLine AnalyticalProcessing** 的简称，意思是**对数据库数据进行实时分析处理**。

为了便于理解，称之为 **窗口函数**。常规的SELECT语句都是对整张表进行查询，而**窗口函数可以让我们有选择的去某一部分数据进行汇总、计算和排序**。

窗口函数的通用形式：

```sql
<窗口函数> OVER ([ PARTITION BY <列名> ]
                     [ ORDER BY <排序用列名> ])
```
[ ]中的内容可以省略。

窗口函数最关键的是搞明白关键字 **PARTITON BY** 和 **ORDER BY** 的作用。

- PARTITON BY 子句 可选参数，指示如何将查询行划分为组，类似于 **GROUP BY 子句的分组功能**，但是 PARTITION BY 子句并**不具备** GROUP BY 子句的**汇总功能**，并**不会改变原始表中记录的行数**。

- ORDER BY 子句 可选参数，指示**如何对每个分区中的行进行排序**，即决定窗口内，是按那种规则(字段)来排序的。

注意
虽然 PARTITON BY 子句 和 ORDER BY 子句 都是可选参数，但是**两个参数不能同时没有（最少二选一）**。不然， <窗口函数> OVER( ) 这种用法没用实际意义（窗口由所有查询行组成，窗口函数使用所有行计算结果）。

```sql
SELECT product_name
       ,product_type
       ,sale_price
       ,RANK() OVER (PARTITION BY product_type
                         ORDER BY sale_price) AS ranking
  FROM product;
```

![jupyter](./images/sql_06_01.png)

### 2 窗口函数种类

大致来说，窗口函数可以分为两类。

一是 将SUM、MAX、MIN等聚合函数用在窗口函数中

二是 RANK、DENSE_RANK等排序用的专用窗口函数

#### 2.1 专用窗口函数

- RANK函数：计算排序时，如果存在相同位次的记录，则会跳过之后的位次。比如有 3 条记录排在第 1 位时：1 位、1 位、1 位、4 位……

- DENSE_RANK函数：同样是计算排序，即使存在相同位次的记录，也不会跳过之后的位次。比如有 3 条记录排在第 1 位时：1 位、1 位、1 位、2 位……

- ROW_NUMBER函数：赋予唯一的连续位次。比如有 3 条记录排在第 1 位时：1 位、2 位、3 位、4 位

```sql
SELECT  product_name,
        product_type,
        sale_price,
        -- 对应下表中的 ranking 列
        RANK() OVER (ORDER BY sale_price) AS ranking,
        -- 对应下表中的 dense_ranking 列
        DENSE_RANK() OVER (ORDER BY sale_price) AS dense_ranking,
        -- 对应下表中的 row_num 列
        ROW_NUMBER() OVER (ORDER BY sale_price) AS row_num
  FROM product;
```
![jupyter](./images/sql_06_02.png)

#### 2.2 聚合函数在窗口函数上的使用

聚合函数在窗口函数中的使用方法和之前的专用窗口函数一样，只是出来的结果是一个累计的聚合函数值。
```sql
SELECT  product_id
       ,product_name
       ,sale_price
       ,SUM(sale_price) OVER (ORDER BY product_id) AS current_sum
       ,AVG(sale_price) OVER (ORDER BY product_id) AS current_avg  
  FROM product;
```

![jupyter](./images/sql_06_03.png)


### 3.窗口函数的的应用 - 计算移动平均

在上面提到，聚合函数在窗口函数使用时，计算的是累积到当前行的所有的数据的聚合。 实际上，还可以指定更加详细的汇总范围。该汇总范围称为 框架 (frame)。

语法
```sql
<窗口函数> OVER (ORDER BY <排序用列名>
                 ROWS n PRECEDING )  
                 
<窗口函数> OVER (ORDER BY <排序用列名>
                 ROWS BETWEEN n PRECEDING AND n FOLLOWING)
```

- PRECEDING（“之前”）， 将框架指定为 “截止到之前 n 行”，加上自身行
- FOLLOWING（“之后”）， 将框架指定为 “截止到之后 n 行”，加上自身行

```sql
SELECT  product_id
       ,product_name
       ,sale_price
       ,AVG(sale_price) OVER (ORDER BY product_id
                               ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) AS moving_avg  
  FROM product;
```
![jupyter](./images/sql_06_04.png)


- 原则上，窗口函数只能在SELECT子句中使用。
- 窗口函数OVER 中的ORDER BY 子句并不会影响最终结果的排序。其只是用来决定窗口函数按何种顺序计算。
